In [15]:
import pandas as pd
from pprint import pprint
from olistlib.db.utils import import_query, connect_db
from sklearn import tree, metrics

In [16]:
# Importando a query
query_path = '/home/marcelo/desenvolvimento/proj_ex/teo_calvo/olist_do_teo/prepara_abt/query02.sql'
query = import_query(query_path)
pprint(query)

('SELECT\n'
 "    '{date}' AS dt_referencia,\n"
 '    T1.*,\n'
 '    coalesce(T2.qtde_vda, 0) AS qtde_vda_futura,\n'
 '    CASE\n'
 '        WHEN coalesce(T2.qtde_vda, 0) = 0\n'
 '        THEN 1\n'
 '        ELSE 0\n'
 '        END\n'
 '        AS flag_churn\n'
 '\n'
 'FROM(\n'
 '\n'
 '    SELECT\n'
 '        T1.seller_id,\n'
 '        max(T1.dt_venda) AS dt_ult_vda,\n'
 '        \n'
 '        /* Totais */\n'
 '        sum(T1.price) AS receita_total,\n'
 '        count(DISTINCT T1.order_id) AS qtde_vdas,\n'
 '        count(T1.product_id) AS qtde_itens_total,\n'
 '        count(DISTINCT T1.product_id) AS qtde_itens_dist_total,\n'
 '        sum(T1.freight_value) AS frete_total,\n'
 '\n'
 '        /* Médias por pedido */\n'
 '        sum(T1.price) / count(DISTINCT T1.order_id) AS receita_por_venda,\n'
 '        count(T1.product_id) / count(DISTINCT T1.order_id) AS '
 'items_por_vda,\n'
 '        sum(T1.freight_value) / count(DISTINCT T1.product_id) AS '
 'frete_por_vda,\n'
 '        sum(T

In [17]:
datas = ['2017-01-01',
        '2017-02-01',
        '2017-03-01',
        '2017-04-01',
        '2017-05-01',
        '2017-06-01',
        '2017-07-01',
        '2017-08-01',
        '2017-09-01'
        ]
print(datas)

['2017-01-01', '2017-02-01', '2017-03-01', '2017-04-01', '2017-05-01', '2017-06-01', '2017-07-01', '2017-08-01', '2017-09-01']


In [18]:
conn = connect_db('sqlite', 
    path='/home/marcelo/desenvolvimento/proj_ex/teo_calvo/olist_do_teo/upload_olist/data/olist.db'
    )

In [19]:
dfs = []
for data in datas:
    query_formatada = query.format(date = data)
    df_tmp = pd.read_sql_query(query_formatada, conn)
    dfs.append(df_tmp)

In [20]:
abt = pd.concat(dfs, axis=0, ignore_index=True)
abt.head()

,dt_referencia,seller_id,dt_ult_vda,receita_total,qtde_vdas,qtde_itens_total,qtde_itens_dist_total,frete_total,receita_por_venda,items_por_vda,frete_por_vda,frete_por_item,proporcao_ativacao,qtde_vda_futura,flag_churn
0,2017-01-01,f09b760d23495ac9a7e00d29b769007c,2016-10-03 16:56:50,21.90,1,1,1,17.19,21.90,1,17.19,17.19,0.166667,0,1
1,2017-01-01,45d33f715e24d15a6ccf5c17b3a23e3c,2016-10-03 21:13:36,36.49,1,1,1,17.24,36.49,1,17.24,17.24,0.166667,17,0
2,2017-01-01,b499c00f28f4b7069ff6550af8c1348a,2016-10-03 22:31:31,29.99,1,1,1,10.96,29.99,1,10.96,10.96,0.166667,10,0
3,2017-01-01,4b1eaadf791bdbbad8c4a35b65236d52,2016-10-04 09:16:33,32.49,1,1,1,11.74,32.49,1,11.74,11.74,0.166667,6,0
4,2017-01-01,624f4ece8da4aafb77699233d480f8ef,2016-10-04 10:16:04,189.00,1,1,1,48.45,189.00,1,48.45,48.45,0.166667,2,0


In [21]:
print(
abt['dt_referencia'].unique())

['2017-01-01' '2017-02-01' '2017-03-01' '2017-04-01' '2017-05-01'
 '2017-06-01' '2017-07-01' '2017-08-01' '2017-09-01']


In [29]:
med_churn_da_base = abt['flag_churn'].mean()

In [23]:
# definição da váriavel target
# definição das features para fazer a previsão
target = 'flag_churn'
features = abt.columns[3:-2]
print(target)
print(features)

flag_churn
Index(['receita_total', 'qtde_vdas', 'qtde_itens_total',
       'qtde_itens_dist_total', 'frete_total', 'receita_por_venda',
       'items_por_vda', 'frete_por_vda', 'frete_por_item',
       'proporcao_ativacao'],
      dtype='object')


In [24]:
#Definição do classificador e seu ajuste
clf = tree.DecisionTreeClassifier(max_depth=10)
clf.fit(abt[features], abt[target])

DecisionTreeClassifier(max_depth=10)

In [25]:
# Criação da base preditiva
y_pred = clf.predict(abt[features])
y_prob = clf.predict_proba(abt[features])

Conceito AUC e ROC:

https://medium.com/@eam.avelar/o-que-%C3%A9-auc-e-roc-nos-modelos-de-machine-learning-2e2c4112033d

In [26]:
# Acurácia
acc = metrics.accuracy_score(abt[target], y_pred)

In [27]:
# ROC
auc = metrics.roc_auc_score(abt[target], y_prob[:,1])

In [28]:
# Váriavel mais importante para o modelo tomar decisão
features_importance = pd.Series(clf.feature_importances_, index=features)
features_importance.sort_values(inplace=True)